In [8]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"var: ")

_set_env("OPENAI_API_KEY")

In [9]:
from openai import OpenAI
from pydantic import BaseModel, Field

client = OpenAI()

class RouterTag(BaseModel):
    input_tag: str = Field(..., description="The tag with a classification of the input to process")
    

def classify(prompt: str) -> str:
    response = client.responses.parse(
        model="gpt-5-mini",
        instructions="You classify tasks into 3 different categories: 'engineering', 'marketing', 'sales'",
        input=prompt,
        text_format=RouterTag
    )
    return response.output_parsed

print(classify("Fix bug with landing page"))
print(classify("Design a new promotional campaign for our latest product"))  # marketing example
print(classify("Reach out to potential clients to close deals"))             # sales example

input_tag='engineering'
input_tag='marketing'
input_tag='sales'


In [ ]:
def engineering_agent(task: str):
    """
    Engineering agent: Specializes in software development, bug fixes, and technical analysis.
    Tools available: codebase access, bug tracker, documentation.
    """
    response = client.responses.create(
        model="gpt-5-mini",
        instructions=(
            "You are an engineering expert, skilled at solving software problems, fixing bugs, "
            "and providing technical solutions. Always use precise technical language, "
            "cite documentation when relevant, and reference relevant code modules. "
            "Your outputs at this first stage are short one paragraph plans for the task."
        ),
        input=task
    )
    return response.output_text

def marketing_agent(task: str):
    """
    Marketing agent: Specializes in creating campaigns, content, and brand strategies.
    Tools available: campaign planner, analytics dashboard, content calendar.
    """
    client = OpenAI()
    response = client.responses.parse(
        model="gpt-5-mini",
        instructions=(
            "You are a marketing expert, focused on campaign creation, brand strategy, "
            "and content ideation. Use persuasive and creative language, and connect ideas "
            "to business objectives. "
            "Your outputs at this first stage are short one paragraph plans for the task."
        ),
        input=task
    )
    return response.output_text

def sales_agent(task: str):
    """
    Sales agent: Specializes in outreach, client management, and closing deals.
    """
    client = OpenAI()
    response = client.responses.create(
        model="gpt-5-mini",
        instructions=(
            "You are a sales expert with skill in outreach, managing leads, "
            "and progressing conversations toward closing deals. "
            "Use persuasive, professional language and reference tools when needed. "
            "Your outputs at this first stage are short one paragraph plans for the task."
        ),
        input=task
    )
    return response.output_text


def router(prompt: str) -> str:
    tag = classify(prompt)
    print("Classified task into: ", tag.input_tag)
    if tag.input_tag == "engineering":
        return engineering_agent(prompt)
    elif tag.input_tag == "marketing":
        return marketing_agent(prompt)
    elif tag.input_tag == "sales":
        return sales_agent(prompt)
    else:
        return "No matching agent found"

# example for an engineering task
print(router("Fix bug with landing page"))
# print(router("Design a new promotional campaign for our latest product"))  # marketing example
# print(router("Reach out to potential clients to close deals"))             # sales example

Classified task into:  engineering
Plan: I’ll first reproduce the failure locally and capture exact symptoms (HTTP status, console & server logs, browser/network trace, screenshots) so we know whether this is a routing, template, asset, CORS/CSP, auth/redirect, or build-cache issue. Next I’ll run the app in debug mode and inspect the landing-page route/controller (e.g., app/routes or pages/index.jsx/tsx, server-side controller, and template files like templates/index.html), the client bundle (webpack/Parcel/Vite build output), middleware (authentication, feature-flag, or A/B tests), and the webserver/proxy config (nginx/CloudFront) to find where the request diverges from expected behavior. I’ll add a minimal failing test or reproduce script, implement a focused fix (route/template/asset-path/redirect/CSP header change or cache invalidation), and unit/integration test the fix (see framework docs: Express routing, Next.js pages, or Django views). Finally I’ll deploy to staging, run smoke